In [1]:
import numpy as np
import sympy
import control as cnt
from sympy import *
from sympy.physics.vector.printing import vlatex
from IPython.display import Math, display

init_printing()

def dp(expr):
    display(Math(vlatex(expr)))

# Finding the Lagrangian

defined as L = K - P, the difference between the kinetic and potential energy of the system

use sympy to create equations for K and P and compute

In [2]:
t = symbols('t') # defining t and theta/phi/other coords as functions of t is necessary to get proper EoMs

x = symbols('x_1', cls=Function)
x = x(t)
x_dot = x.diff(t)
x_ddot = x.diff(t,2)

b, g, m, k1, k2, F, h, s, c = symbols('b, g, m, k_1, k_2, F, h, s, c')

linear velocities...

In [3]:
q = Matrix([[x], [0], [0]]) # the z coord will be used by our angular velocity, here it's just 0
q_dot = q.diff(t)
dp(q_dot)
v = q_dot

<IPython.core.display.Math object>

now it's time for angular...

In [4]:
w = Matrix([[0], [0], [0]]) # all zeros, linear movement only

# potential and kinetic energy

kinetic is given by $ K = {1/2 m_1 v_1^T v_1 + 1/2\omega_1^T J_1 \omega_1}$ for however many things you have

potential is $ P = {m_1 g h_1} $ + whatever springs are present

J matrices...

point mass: $ J = 0 $ :steamhappy:

thin rod: $ J = \begin{pmatrix}
0 & 0 & 0\\
0 & \frac{m l ^2}{12} & 0\\
0 & 0 & \frac{m l ^2}{12}
\end{pmatrix} $

thin plate: $ J = \begin{pmatrix}
\frac{m \omega ^2}{12} & 0 & 0\\
0 & \frac{m l ^2}{12} & 0\\
0 & 0 & \frac{m l ^2}{12} + \frac{m \omega ^2}{12}
\end{pmatrix} $

In [5]:
K_total = 1/2*m*v.T @ v
# dp(K_total)
K = 0.5*m*x_dot**2
# dp(K)
P_spring = 1/2*k1*x**2 + 1/4*k2*x**4
P = m*g*(h - x) - P_spring
L = K - P
dp(L)

<IPython.core.display.Math object>

# lagrangian

q and q_dot are for our generalized coordinates (in the practice final it's just theta and theta_dot, for 2 gen coords it will be different)

In [6]:
# behold...
def derive_Lagrangian(L, q, q_dot):
    term_1 = (sympy.tensor.derive_by_array(L, q_dot)).diff(t)
    term_2 = sympy.tensor.derive_by_array(L, q)
    return term_1 - term_2

# def derive_Lagrangian(L, z, theta, z_dot, theta_dot):
#     term_1 = (sympy.tensor.derive_by_array(L, z_dot)).diff(t)
#     term_2 = sympy.tensor.derive_by_array(L, z)
#     term_3 = (sympy.tensor.derive_by_array(L, theta_dot)).diff(t)
#     term_4 = sympy.tensor.derive_by_array(L, theta)
#     ans = Matrix([term_1 - term_2, term_3 - term_4])
#     return ans

In [7]:
LG = derive_Lagrangian(L, x, x_dot)
dp(LG)

<IPython.core.display.Math object>

# Euler Lagrange

defined by 
# $ \frac{d}{dt} (\frac{\delta L (q, \dot{q})}{\delta{\dot{q}}}) - \frac{\delta L (q)}{\delta q} = \tau - B \dot{q}$

n = number of gen coords

$ \dot{q}$ is the dot for all n gen coords, should be n x 1

$ B $ is the damping force for each gen coord, should be n x n

$ \tau $ has a force for each spatial gen coord and a torque for each angular gen coord, should be n x 1 

In [8]:
LHS = Matrix([[LG]])

Tau = Matrix([[F]])
B = Matrix([[b + c*sign(x_dot)]])
q_d = Matrix([[x_dot]])
RHS = Tau - B @ q_d

dynamics = Eq(LHS, RHS) # this is also our euler lagrange eq
dp(dynamics)

<IPython.core.display.Math object>

# Equations of motion

taking $ \begin{pmatrix}
        \theta\\
        \dot{\theta}        
        \end{pmatrix} $ and making it into 
        $ \begin{pmatrix}
        \dot{\theta}\\
        \ddot{\theta}       
        \end{pmatrix} $

essentially we solve for $ \ddot{\theta} $ and plug it into another matrix with $ \dot{\theta} $

inside the `(theta_ddot,)` we can include however many variables we need to solve for, namely however many ddots are involved

In [9]:
solve_dict = solve(dynamics, (x_ddot,), simplify=True, dict=True)[0]
# dp(solve_dict)

x_ddot_expr = solve_dict[x_ddot]

EoM = Matrix([x_dot, x_ddot_expr])
dp(EoM)

<IPython.core.display.Math object>

# Part 2
# Linearizing and stuff

first step of linearization is finding the equilibria, or where the system is at rest/balanced

get $ \theta_e $ and then find the $ \tau_e $ to maintain $ \theta_e $

at equilibrium $ \theta = 0 $ so $ \dot{\theta} $ and $ \ddot{\theta} $ also $ = 0 $ so we can plug in $ 0 $ for those and solve for tau   

In [10]:
# part 2 init
z = symbols('z', cls=Function)
z = z(t)
z_dot = z.diff(t)
z_ddot = z.diff(t,2)

ze = symbols('z_e', cls=Function)
ze = ze(t)
ze_dot = ze.diff(t)
ze_ddot = ze.diff(t,2)

Fe = symbols('F_e')

LHS2 = m*z_ddot + k1*z + k2*z**3 - 1/sqrt(2)*m*g
RHS2 = F- b*z_dot
EoM_p2 = Eq(LHS2, RHS2)

dp(EoM_p2)

<IPython.core.display.Math object>

In [11]:
subs_dict = {b: 0.1, m: 0.5, k1: 0.05, k2: 0.02, g: 9.8}
subs_eq = {F: Fe, z: ze, z_dot: 0}
subs_dict_eq = {F: Fe, z: ze, z_dot: ze_dot, b: 0.1, m: 0.5, k1: 0.05, k2: 0.02, g: 9.8}

In [12]:
# 2.1
EoM_p2_eq = EoM_p2.subs(subs_eq)
F_e = solve(EoM_p2_eq.subs({ze: 0}), (Fe,), simplify = True, dict = True)[0] # the subs here is to set theta_e = 0, it isn't done for EoM_eq for symbolic porpoises
dp(F_e)

<IPython.core.display.Math object>

In [13]:
# 2.2 dynamics stuff
solve_dict_p2 = solve(EoM_p2, (z_ddot,), simplify=True, dict=True)[0]
z_ddot_expr = solve_dict_p2[z_ddot]
dynamics_p2 = Matrix([[z_dot], [z_ddot_expr]])
dp(dynamics_p2)

<IPython.core.display.Math object>

In [14]:
# 2.3 linearizing
ze_tilde, ze_tilde_dot, ze_tilde_ddot, z_tilde, z_tilde_dot, z_tilde_ddot = symbols('\\tilde{z_e}, \\dot{\\tilde{z_e}}, \\ddot{\\tilde{z_e}}, \\tilde{z}, \\dot{\\tilde{z}}, \\ddot{\\tilde{z}}') 
F_tilde = symbols('\\tilde{F}')
linear_dict = {z: ze + z_tilde, z_dot: 0 + z_tilde_dot, z_ddot: 0 + z_tilde_ddot, F: F_tilde}

LHS2_lin = LHS2.subs(linear_dict)
RHS2_lin = RHS2.subs(linear_dict)

dynamics_p2_lin = Eq(LHS2_lin, RHS2_lin)
dp(dynamics_p2_lin)

<IPython.core.display.Math object>

we want

$ \dot{x} = Ax + B\tilde{u} $

$ y = Cx + D\tilde{u} $

note: these are the un-equilibrized (not a word) equations because otherwise it won't work. there's probably a work around but it's 10pm and i don't want to work any harder on this than i have to. i will `.subs({theta: 0})` later on for the transfer function

In [15]:
dynamics_p24 = dynamics_p2_lin.subs({ze: 0})
dp(dynamics_p24)

solve_p24 = solve(dynamics_p24, (z_tilde_ddot,), simplify=True, dict=True)[0]
z_tilde_ddot_expr = solve_p24[z_tilde_ddot]

EoM_p24 = Matrix([[z_tilde_dot],[z_tilde_ddot_expr]])
dp(EoM_p24)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [19]:
# define state variables
state = Matrix([z_tilde, z_tilde_dot])

A = EoM_p24.jacobian(state)

dp(A.subs({z_tilde: 0}))

<IPython.core.display.Math object>

In [20]:
inputs = Matrix([[F_tilde]])

B = EoM_p24.jacobian(inputs)
dp(B)

<IPython.core.display.Math object>

We still need C and D equations, 

$ y = Cx + Du $

$C$ is used to select the variables present in your output $y$, it will be a 1 x n where $x$ is a n x 1

$D$ is normally zero... so stick with that I guess

In [21]:
C = Matrix([[1, 0]])
D = Matrix([0])

# C_eq and D_eq aren't going to be any different...

we can use these matrices to write our linearized state space equations, and find our transfer function

Now we get to use the control library which makes life 10000000000000000x better!!!!

plug in real values into your A B C and D matrices and pass them into `cnt.StateSpace()`

In [75]:
mass_subs = subs_dict

A_sub = A.subs({z_tilde: 0})
A_sub2 = A_sub.subs(mass_subs)
B_sub = B.subs(mass_subs)
C_sub = C
D_sub = D
A_sub2

mass = cnt.ss(A_sub2, B_sub, C_sub, D_sub, False)

dp(B_sub)

<IPython.core.display.Math object>


now do `cnt.ss2tf` (state space to transfer func)

In [76]:
cnt.ss2tf(mass)

TransferFunction(array([2.]), array([1. , 0.2, 0.1]))

i'm not sure why that s term is in the top. it's  .0000000000000035553 so we're just going to ignore it... ok?

we can also put the entire system in canonical observable or controller form with a single function

we find our transfer function symbolically with the following equation:

$ H(s) = C(sI-A)^{-1}B+D $

$I$ is n x n (same as $A$)

In [36]:
I = eye(2)

H = C.subs({z_tilde: 0}) @ (s*I - A.subs({z_tilde: 0})).inv() @ B.subs({z_tilde: 0}) + D.subs({z_tilde: 0})

dp(H)

<IPython.core.display.Math object>

add a kd*s and kp to the denominator, then divide by ml**2 resultant fraction for gain stuff should be:

In [50]:
kp, kd = symbols('k_p, k_d')
top = 1/(m)
bottom = (kp + k1)/(m) + s*(kd + b)/(m) + s**2

tf_gains = top/bottom
tf_gains_sub = tf_gains.subs(mass_subs)
dp(tf_gains_sub)

<IPython.core.display.Math object>

# Finding the gains

In [62]:
wn = symbols('\\omega_n')

zeta_val = 0.707
kp_val = 5/1

wn_eq = sqrt((kp_val + k1)/(m))
wn_val = wn_eq.subs(mass_subs)

kd_eqtn = Eq(2*wn*zeta_val,(b+kd)/(m)) # change the RHS to 

kd_val = solve(kd_eqtn.subs(mass_subs), kd)[0]


dp(kd_val)
# kd_val.subs({wn: wn_val})

# tf_gains_sub.subs({kd: kd_val, kp: kp_val, wn: 3.178})

<IPython.core.display.Math object>

with this we can directly solve for kd and wn

In [ ]:
cnt.canonical_form(rodmass, 'reachable') 

# the string tells it what canonical form to find.
# observable is observer canonical form
# reachable is controller canonical form
# modal is some other thing we never went over, don't use it

there are once again really tiny values in this, no doubt from the $ 3.553 \times 10^{-15} $ term... i'm just evaluating it at 0

producing controllability and observability matrices

In [ ]:
cnt.ctrb(A_sub, B_sub)
cnt.obsv(A_sub, B_sub)

# PID and PD Controllers

In [ ]:
# PD setup

def __init__(self):
        tr = 0.699
        zeta = 0.707

        # desired natural frequency
        wn = 2.2 / tr
        alpha1 = 2.0 * zeta * wn
        alpha0 = wn**2

        b0 = 
        a0 =            # solve for these
        a1 =
        self.kp = (alpha0 - a0) / b0
        self.kd = (alpha1 - a1) / b0
        # PD gains
        print('kp: ', self.kp)
        print('kd: ', self.kd)

alternative init method with poles instead of wn, tr, and zeta

In [ ]:
    def __init__(self):
        # Declare pole locations
        p1 = -1
        p2 = -1.5
        alpha0 = (-p1) * (-p2) # Coefficient in front of s^0 in desired characteristic equation
        alpha1 = (-p1) + (-p2) # Coefficient in front of s^1 in desired characteristic equation
        b0 = 1/P.m
        a0 = P.k/P.m
        a1 = P.b/P.m
        self.kp = (alpha0 - a0) / b0
        self.kd = (alpha1 - a1) / b0
        # PD gains
        print('kp: ', self.kp)
        print('kd: ', self.kd)

In [ ]:
# PD update functions have the form:

def update(self, theta_r, x):
        theta = x[0][0]
        thetadot = x[1][0]
        # compute the linearized torque using PD control
        tau_tilde = self.kp * (theta_r - theta) - self.kd * thetadot

        tau_sat = saturate(tau_tilde, P.F_max)
        return tau_sat

# saturation isn't as

In [ ]:
# PID setup
def __init__(self):
        #  tuning parameters
        tr = 2.0
        zeta = 0.707
        self.ki = 1.5  # integrator gain, tune this

        # desired natural frequency
        wn = 2.2 / tr
        alpha1 = 2.0 * zeta * wn
        alpha0 = wn**2

        b0 = 
        a0 =            # solve for these
        a1 = 
        self.kp = (alpha0 - a0) / b0
        self.kd = (alpha1 - a1) / b0
        print('kp: ', self.kp)
        print('ki: ', self.ki)
        print('kd: ', self.kd)

        # dirty derivative gainss
        self.beta = (2.0 * P.sigma - P.Ts) / (2.0 * P.sigma + P.Ts)  # dirty derivative gain

        #----------------------------------------------------------
        # variables for integrator and differentiator
        self.z_dot = 0.0             # estimated derivative of y
        self.z_d1 = 0.0              # Signal y delayed by one sample
        self.error_dot = 0.0         # estimated derivative of error
        self.error_d1 = 0.0          # Error delayed by one sample
        self.integrator = 0.0        # integrator

In [ ]:
# PID updates are like this:
def update(self, z_r, y):
        z = y[0][0]
        
        # Compute the current error
        error = z_r - z

        # integrate error
        self.integrator = self.integrator + (P.Ts / 2) * (error + self.error_d1)

        # differentiate z
        self.z_dot = self.beta * self.z_dot + (1 - self.beta) * ((z - self.z_d1) / P.Ts)
        
        # PID control
        F_unsat = self.kp * error + self.ki * self.integrator - self.kd * self.z_dot
        F = saturate(F_unsat, P.F_max)

        # integrator anti - windup
        if self.ki != 0.0:
            self.integrator = self.integrator + P.Ts / self.ki * (F - F_unsat)
            
        # update delayed variables
        self.error_d1 = error
        self.z_d1 = z
        return F

In [ ]:
# saturation function in case you lose it

def saturate(self, u):
    if abs(u) > self.limit:
        u = self.limit*np.sign(u)
    return u

# also anti-windup just in case

if np.abs(self.h_dot) < 0.5:
    self.integrator_h = self.integrator_h + (P.Ts / 2) * (error_h + self.error_h_d1)



# observer controllers
these update both a PID controller and an observer so there's 2 update functions 

this code is for a disturbance observer. idk the difference between this and just a regular observer so... cool

In [ ]:
class ctrlObserver:
    def __init__(self):
        #--------------------------------------------------
        # State Feedback Control Design
        #--------------------------------------------------
        #  tuning parameters
        tr = 1.5
        zeta = 0.707
        integrator_pole = -1
        tr_obs = tr/10  # rise time frequency for observer
        zeta_obs = 0.707  # damping ratio for observer


        # State Space Equations
        # xdot = A*x + B*u
        # y = C*x
        self.A = np.array([[0.0, 0.0],
                           [0.0, 0.0]])
        self.B = np.array([[0.0],
                           [0.0]])       
        self.C = np.array([[0.0, 0.0]])
        # form augmented system
        A1 = np.vstack((np.hstack((self.A, np.zeros((2,1)))), # np.zeros matches A
                        np.hstack((-self.C, np.zeros((1,1)))) )) #np.zeros matches C
        B1 = np.vstack( (self.B, 0.0) )

        # gain calculation
        wn = 2.2 / tr  # natural frequency
        des_char_poly = np.convolve([1, 2*zeta*wn, wn**2],
                                    [1, -integrator_pole])
        des_poles = np.roots(des_char_poly)
        # Compute the gains if the system is controllable
        if np.linalg.matrix_rank(cnt.ctrb(A1, B1)) != 3: # rank will be n + 1, where A is a n x n matrix
            print("The system is not controllable")
        else:
            K1 = cnt.place(A1, B1, des_poles)
            self.K = K1[0][0:2]
            self.ki = K1[0][2]


        # observer design
        wn_obs = 2.2 / tr_obs
        des_obsv_char_poly = [1, 2*zeta_obs*wn_obs, wn_obs**2]
        des_obsv_poles = np.roots(des_obsv_char_poly)
        # Compute the gains if the system is controllable
        if np.linalg.matrix_rank(cnt.ctrb(self.A.T, self.C.T)) != 2:
            print("The system is not observerable")
        else:
            self.L = cnt.acker(self.A.T, self.C.T, des_obsv_poles).T
        print('K: ', self.K)
        print('ki ', self.ki)
        print('L^T: ', self.L.T)
        #--------------------------------------------------
        # variables to implement integrator
        self.integrator = 0.0  # integrator
        self.error_d1 = 0.0  # error signal delayed by 1 sample
        self.x_hat = np.array([
            [0.0],  # z_hat_0
            [0.0],  # zdot_hat_0
        ])
        self.tau_d1 = 0.0  # control torque, delayed 1 sample

    def update(self, z_r, y):
        # update the observer and extract z_hat
        x_hat = self.update_observer(y)
        z_hat = x_hat[0][0]
        # integrate error
        error = z_r - z_hat
        self.integrator = self.integrator \
                          + (P.Ts / 2.0) * (error + self.error_d1)
        self.error_d1 = error
        # Compute the state feedback controller
        f_tilde = -self.K @ x_hat - self.ki * self.integrator
        # compute total torque
        F = saturate(f_tilde[0], P.F_max)
        self.tau_d1 = F
        return F, x_hat

    def update_observer(self, y_m):
        # update the observer using RK4 integration
        F1 = self.observer_f(self.x_hat, y_m)
        F2 = self.observer_f(self.x_hat + P.Ts / 2 * F1, y_m)
        F3 = self.observer_f(self.x_hat + P.Ts / 2 * F2, y_m)
        F4 = self.observer_f(self.x_hat + P.Ts * F3, y_m)
        self.x_hat = self.x_hat + P.Ts / 6 * (F1 + 2*F2 + 2*F3 + F4)
        return self.x_hat

    def observer_f(self, x_hat, y_m):
        # xhatdot = A*(xhat-xe) + B*(u-ue) + L(y-C*xhat)
        xhat_dot = self.A @ x_hat\
                   + self.B * (self.tau_d1)\
                   + self.L * (y_m - self.C @ x_hat)
        return xhat_dot


def saturate(u, limit):
    if abs(u) > limit:
        u = limit * np.sign(u)
    return u

# finding feedback gains

the augmented feedback gain $K$ is defined by this equation:

$ K = (\alpha - \textbf{a}_A) \mathcal{A}_A^{-1} \mathcal{C}_{AB}^{-1}$

$\alpha$ is an array of the n-1, n-2, etc coefficients of $ \Delta_{cl}^{d} $ (desired close loop char polynom)

you can find $ \Delta_{cl}^{d} $ by foiling out the desired poles

$ \textbf{a}_A $ is an array of the n-1, n-2, n-3 etc coefficients of $ \Delta_{ol} $ (open loop char polynom) 

you can find $ \Delta_{ol} = \text{det} (sI - A)$

(skip the coefficients of the highest power s terms of the characteristic polynomials. that's what i mean by n-1, n-2 coeff but i want to be very clear)


$\mathcal{A}_A$ is a matrix of the form 

$ \begin{pmatrix}
1 & a & b\\
0 & 1 & a\\
0 & 0 & 1
\end{pmatrix} $

where $ a_A = \begin{pmatrix}
a & b & c
\end{pmatrix} $


you will have to `.inv` $ \mathcal{A}_A $ to get $\mathcal{A}_A^{-1}$


$\mathcal{C}_{AB}^{-1}$ is your controllability matrix. use `cnt.ctrb(A,B)` to find $\mathcal{C}_{AB}$ and then .inv it


# note

feedback gain that places poles at desired location is given by $ K_c = \alpha - \textbf{a}_A $ but this is only when in controller canonic form

# reference gain

$ k_r = \frac{-1}{C_r (A-BK)^{-1}B} $

$C_r$ is the $C$ matrix required to for the reference output, a la $y_r = C_r x$

In [66]:
simplify((s-2+2j)*(s-2-2j))

In [71]:
K1, K2 = symbols('K_1, K_2')
K = Matrix([[K1, K2]])
des_cl_bk = det(s*I - (A_sub2 - B_sub @ K))
dp(des_cl_bk)


<IPython.core.display.Math object>

In [112]:
# augmenting

Cr = Matrix([[1.0, 0.0]])

A_1 = np.vstack((np.hstack((A_sub2, np.zeros((np.size(A_sub2,1),1)))), 
                        np.hstack((-Cr, np.array([[0.0]])))))
B_1 = np.vstack((B_sub, 0.0))

ctrb = cnt.ctrb(A_sub2, B_sub)
obsv = cnt.obsv(A_sub2, C_sub)

ctrb
ctrb_np = np.array([[0.,2.],[2.,-0.4]])

A_1

A_1_np = np.array([[0.0, 1.0, 0.0], [-0.1, -0.2, 0,0], [-1.0, 0.0, 0.0]])

array([[0, 1, 0.0],
       [-0.100000000000000, -0.200000000000000, 0.0],
       [-1.00000000000000, 0, 0.0]], dtype=object)

In [81]:
ol_char = det(s*eye(3) - A_1)
dp(ol_char)

<IPython.core.display.Math object>

In [111]:
a_A = np.array([[0.2, 0.1, 0]])

A_A = np.array([[1, 0.2, 0.1],
                [0, 1, 0.2],
                [0, 0, 1]])

cl_des_char = (s - 2 + 2j)*(s - 2 - 2j)
# dp(simplify(cl_des_char))

alpha = np.array([[-4, 8, 0]])

L = (alpha - a_A) @ np.linalg.inv(A_1_np) @ np.linalg.inv(ctrb_np)

A_K = np.array([[0, 1],[-.1, -.2]])
B_K = np.array([[0],[2]])

p1 = -2.0 + 2.0j
p2 = -2.0 - 2.0j
pi = -5.0

K = cnt.place(A_K, B_K, [p1, p2])

Kl = cnt.acker(A_1, B_1, [p1, p2, pi])

K

UFuncTypeError: Cannot cast ufunc 'inv' input from dtype('O') to dtype('float64') with casting rule 'same_kind'

In [ ]:
np.linalg.eigvals(A_K - )

# finding observer gains

augmented observer gain defined as 

$ L = \mathcal{O}_{A,C}^{-1} \mathcal{A}_{A}^{-T} (\beta - \textbf{a}_A)^{T} $

$ \mathcal{O}_{A,C} $ is your observability matrix, use `cnt.obsv(A,B)` to find it
$ \beta $ is an array of the n-1, n-2, etc coefficients of $ \Delta_{cl,obs}^{d} $ (desired close loop char polynom)

$ \Delta_{cl,obs}^{d} = \text{det}(sI - (A_o - L_o C_o))$

$ \textbf{a}_A $ is an array of the n-1, n-2, n-3 etc coefficients of $ \Delta_{ol} $ (open loop char polynom) 

$ \Delta_{ol} = \text{det}(sI - A_o) $

once again, $\mathcal{A}_A$ is a matrix of the form 

$ \begin{pmatrix}
1 & a & b\\
0 & 1 & a\\
0 & 0 & 1
\end{pmatrix} $

where $ a_A = \begin{pmatrix}
a & b & c
\end{pmatrix} $

you will have to take the transpose, inv transpose, or inv of some things to make it work right


# notes
observer gain $ L_o $ that places the poles at the desired locations is given by: $ A_o − L_o C_o $ but this is only when in observer canonic form

`o` subscript gains mean the array is in observer canonical form